# Train Models


## Connect to your workspace

To get started, connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [1]:
pip install argparse

Note: you may need to restart the kernel to use updated packages.


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
               
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.38.0 to work with machinelearningfinal


# Prepare data

In [3]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'house prices dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/over 5 years.csv'], # Upload the diabetes csv files in /data
                        target_path='house-prices-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'house-prices-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='house prices dataset',
                                description='house prices data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


# Create scripts for pipeline steps

In [4]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'house_prediction_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

house_prediction_pipeline


## Prepare compute env

## Prep and training steps

In [5]:
%%writefile $experiment_folder/prep_houses.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
houses = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(houses))
run.log('raw_rows', row_count)

# remove nulls
houses = houses.dropna()

# Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['nombre_pieces_principales','surface_terrain','surface_reelle_bati']
houses[num_cols] = scaler.fit_transform(houses[num_cols])

# Log processed rows
row_count = (len(houses))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
houses.to_csv(save_path, index=False, header=True)

# End the run
run.complete()

Overwriting house_prediction_pipeline/prep_houses.py


In [6]:
%%writefile $experiment_folder/houses_pricing-training.py
# Import libraries
from azureml.core import Run,Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--training-data", type=str, dest='training_data', help='training data')
args = parser.parse_args()
training_data = args.training_data

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
file_path = os.path.join(training_data,'data.csv')
houses = pd.read_csv(file_path)

# Separate features and predicted result
X = houses[['nombre_pieces_principales','surface_terrain','surface_reelle_bati']]
y = houses['valeur_fonciere']

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)


model = LinearRegression().fit(X_train, y_train)

# calculate r2
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred, multioutput='variance_weighted')
print('R2: ' + str(r2))
run.log('R2', np.float(r2))

# calculate mse
mse = mean_squared_error(y_test, y_pred)
print('MSE:' + str(mse))
run.log('MSE', np.float(mse))

# calculate mae (mean absolute error)
mae = mean_absolute_error(y_test, y_pred)
print('MAE:' + str(mae))
run.log('MAE', np.float(mae))

# Save the trained model in the outputs folder
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'house_prices_model.pkl')
joblib.dump(value=model, filename=model_file)
# Register the model
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'house-prices_model',
               tags={'Training context':'Pipeline'},
               properties={'R2': np.float(r2), 'MSE': np.float(mse), 'MAE': np.float(mae)})



Overwriting house_prediction_pipeline/houses_pricing-training.py


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [8]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Overwriting house_prediction_pipeline/experiment_env.yml


In [9]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'experiment_env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


## Create and run a pipeline

In [10]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
houses_ds = ws.datasets.get("house prices dataset")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
prep_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_houses.py",
                                arguments = ['--input-data', houses_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "houses_pricing-training.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [11]:
from azureml.core import Experiment
from azureml.pipeline.core import RegressionPipeline, Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline=Pipeline(memory=None,steps=[('datatransformer', DataTransformer(
    enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, 
    feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, 
    is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta'), 
    
    random_state=0, reg_alpha=2.3958333333333335, reg_lambda=1.7708333333333335, subsample=1, tree_method='auto'))], verbose=False))
    
    RegressionPipeline(pipeline, ('17', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('decisiontreeregressor', DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None, max_features=0.9, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.00218714609400816, min_samples_split=0.0037087774117744725, min_weight_fraction_leaf=0.0, presort='deprecated', random_state=None, splitter='best'))], verbose=False)), ('12', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('extratreesregressor', ExtraTreesRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae', max_depth=None, max_features=0.8, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01091729022778783, min_samples_split=0.001529732116091358, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False))], weights=[0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.26666666666666666, 0.4666666666666667, 0.06666666666666667]))],
            verbose=False),
            stddev=[2472902.6566388463])                   
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'mslearn-house-price_prediction-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

SyntaxError: invalid syntax (<ipython-input-11-b0f621b5caa1>, line 10)

## Publish the pipeline

In [ ]:
# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="house-prices-prediction-training-pipeline", description="Trains houses prices model", version="1.0")

published_pipeline

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")
import requests

experiment_name = 'mslearn-house-pipeline'

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
published_pipeline_run.wait_for_completion(show_output=True)

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-house-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name='mslearn-house-pipeline', 
                                  recurrence=recurrence)
print('Pipeline scheduled.')
schedules = Schedule.list(ws)
pipeline_experiment = ws.experiments.get('mslearn-house-pipeline')
latest_run = list(pipeline_experiment.get_runs())[0]

latest_run.get_details()

You can retrieve the metrics and outputs from the **Run** object.

In [ ]:
for run in pipeline_run.get_children():
    print(run.name, ':')
    metrics = run.get_metrics()
    for metric_name in metrics:
        print('\t',metric_name, ":", metrics[metric_name])

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

In [ ]:
# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="house-training-pipeline", description="Trains house model", version="1.0")

published_pipeline

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

In [ ]:
import requests

experiment_name = 'mslearn-house-prices-pipeline'

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

In [ ]:
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
published_pipeline_run.wait_for_completion(show_output=True)

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-houses-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name='mslearn-houses-pipeline', 
                                  recurrence=recurrence)
print('Pipeline scheduled.')

In [ ]:
schedules = Schedule.list(ws)
schedules

In [ ]:
pipeline_experiment = ws.experiments.get('mslearn-houses-pipeline')
latest_run = list(pipeline_experiment.get_runs())[0]

latest_run.get_details()